# Testing the `models` module

**Authorship:**
Adam Klie, *10/04/2022*
***
**Description:**
Notebook for testing out the `models` module.

In [1]:
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

import os
import torch
import numpy as np
import pandas as pd
import eugene as eu

Global seed set to 13


GPU is available: True
Number of GPUs: 1
Current GPU: 0
GPUs: Quadro RTX 5000


In [ ]:
SEQ_LEN = 66
OUT_DIMS = 2
MODEL = "hybrid"
STRAND = "ss"
TASK = "regression"
LOSS_FXN = "mse"

In [ ]:
# Length of strand
x_len = 66

# Substitute x and x_rev for their respective data
x = torch.randn(10, 4, x_len)
x_rev = torch.randn(10, 4, x_len)
y = torch.randn(10, 2)

In [ ]:
sdata = eu.datasets.random1000()
sdata

# BaseModel

# Base Models

In [ ]:

CNN_KWARGS=dict(channels=[4, 16, 32], 
                conv_kernels=[15, 5], 
                pool_kernels=[1, 1])
RNN_KWARGS=dict(output_dim=32,
                bidirectional=True,
                batch_first=True)
FCN_KWARGS=dict(hidden_dims=[50])

In [ ]:
eu.models.FCN(
    input_len=66
    

In [ ]:
eugene = eu.models.Hybrid(
    input_len=SEQ_LEN, 
    output_dim=OUT_DIMS,
    strand=STRAND,
    task=TASK,
    loss_fxn=LOSS_FXN,
    conv_kwargs=CNN_KWARGS,
    rnn_kwargs=RNN_KWARGS,
    fc_kwargs=FCN_KWARGS
)
eu.models.base.init_weights(eugene)
eugene

# SOTA Models

## DeepBind

In [5]:
simple_deepbind_instance = DeepBind(input_len=x_len)

simple_deepbind_instance

DeepBind(
  (max_pool): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (avg_pool): AvgPool1d(kernel_size=(4,), stride=(4,), padding=(0,))
  (convnet): BasicConv1D(
    (module): Sequential(
      (0): Conv1d(4, 16, kernel_size=(4,), stride=(1,))
      (1): ReLU()
      (2): Dropout(p=0.2, inplace=False)
    )
  )
  (fcn): BasicFullyConnectedModule(
    (module): Sequential(
      (0): Linear(in_features=504, out_features=256, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.2, inplace=False)
      (3): Linear(in_features=256, out_features=64, bias=True)
      (4): ReLU()
      (5): Dropout(p=0.2, inplace=False)
      (6): Linear(in_features=64, out_features=16, bias=True)
      (7): ReLU()
      (8): Dropout(p=0.2, inplace=False)
      (9): Linear(in_features=16, out_features=4, bias=True)
      (10): ReLU()
      (11): Dropout(p=0.2, inplace=False)
      (12): Linear(in_features=4, out_features=1, bias=True)
    )
  )
  (r_squared): R2Score()
)

In [6]:
# x_rev does not need to be specified in single strand mode
out = simple_deepbind_instance(x)

out, out.shape

(tensor([[0.4146],
         [0.4225],
         [0.4162],
         [0.4346],
         [0.4200],
         [0.4334],
         [0.4164],
         [0.4181],
         [0.4171],
         [0.4255]], grad_fn=<AddmmBackward0>),
 torch.Size([10, 1]))

# DeepBind Parameters

- input_len : int - length of input strand
- strand : string - type of strand to process, only  || default : "ss" || res: single ("ss"), double ("ds"), twin ("ts")
- task : string - task for the model to perform || default : "regression" || res: "regression", "binary_classification"

#### conv_kwargs

- channels : list of ints - amount of channels for each convolutional layer || default : [4, 16]
- conv_kernels : list of int(s) - size of convolutional kernels || default : [4] || res : length must be 1 less than that of channels
- pool_kernels : list of int(s) - size of max pooling kernels || default : [4] || res: length must be 1 less than that of channels, only applies is length of channels is > 2
- omit_final_pool : boolean - bypass final max pooling step of output || default : True
- dropout_rates : float - probability for the dropout of any given node || default : 0.2
- batchnorm : boolean - enable batch normalization between layers || default : False

#### mp_kwargs

- kernel_size : int - motif pooling kernel size || default : 4 || res : multiples of 2

#### fc_kwargs

- output_dim : int - number of output dimensions for the network || default : 1
- hidden_dims : list of int(s) - width of each hidden dimension || default : [256, 64, 16, 4]
- dropout_rate : float - probability for the dropout of any given node || default : 0.2
- batchnorm : boolean - enable batch normalization between layers || default : False

In [7]:
customized_deepbind_instance = DeepBind(
    input_len=x_len,
    strand="ts",
    mp_kwargs=dict(kernel_size=16),
    conv_kwargs=dict(channels=[4, 16, 32], conv_kernels=[4, 4], pool_kernels=[4, 4], omit_final_pool=False, dropout_rates=0.5, batchnorm=True),
    fc_kwargs=dict(output_dim=5, hidden_dims=[256, 128, 64, 32, 16], dropout_rate=0.3, batchnorm=True)
)

customized_deepbind_instance

DeepBind(
  (max_pool): MaxPool1d(kernel_size=16, stride=16, padding=0, dilation=1, ceil_mode=False)
  (avg_pool): AvgPool1d(kernel_size=(16,), stride=(16,), padding=(0,))
  (convnet): BasicConv1D(
    (module): Sequential(
      (0): Conv1d(4, 16, kernel_size=(4,), stride=(1,))
      (1): ReLU()
      (2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
      (3): Dropout(p=0.5, inplace=False)
      (4): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): Conv1d(16, 32, kernel_size=(4,), stride=(1,))
      (6): ReLU()
      (7): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
      (8): Dropout(p=0.5, inplace=False)
      (9): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (fcn): BasicFullyConnectedModule(
    (module): Sequential(
      (0): Linear(in_features=12, out_features=256, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.3, inplace=Fals

In [8]:
# x_rev must be specified in double and twin strand modes
out = customized_deepbind_instance(x, x_rev)

out, out.shape

(tensor([[-0.3034],
         [ 0.1875],
         [ 0.0155],
         [ 0.1738],
         [-0.1377],
         [ 0.0516],
         [-0.2159],
         [-0.0738],
         [ 0.0833],
         [ 0.0272]], grad_fn=<UnsqueezeBackward0>),
 torch.Size([10, 1]))

# Simple usage of DeepSEA module

The DeepBind module only requires one parameter to function. It can be further customized with optional parameters.

In [4]:
simple_deepsea_instance = DeepSEA(input_len=x_len)

simple_deepsea_instance

DeepSEA(
  (module): Sequential(
    (0): Conv1d(4, 320, kernel_size=(8,), stride=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.2, inplace=False)
    (4): Conv1d(320, 480, kernel_size=(8,), stride=(1,))
    (5): ReLU()
    (6): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (7): Dropout(p=0.2, inplace=False)
    (8): Conv1d(480, 960, kernel_size=(8,), stride=(1,))
    (9): ReLU()
    (10): Dropout(p=0.2, inplace=False)
  )
  (fcnet): BasicFullyConnectedModule(
    (module): Sequential(
      (0): Linear(in_features=50880, out_features=1, bias=True)
    )
  )
  (r_squared): R2Score()
)

In [5]:
# x_rev does not need to be specified in single strand mode
out = simple_deepsea_instance(x)

out, out.shape

(tensor([[ 0.0167],
         [ 0.0913],
         [ 0.0399],
         [-0.0184],
         [ 0.1422],
         [-0.0843],
         [-0.0960],
         [ 0.0060],
         [ 0.0312],
         [ 0.3167]], grad_fn=<AddmmBackward0>),
 torch.Size([10, 1]))

# Custom Models

# Instantiate a `Jores21CNN` model

In [89]:
model = eu.models.Jores21CNN(170, 1)
model

Jores21CNN(
  (hp_metric): R2Score()
  (biconv): BiConv1D()
  (conv): Conv1d(128, 128, kernel_size=(13,), stride=(1,), padding=same)
  (dropout): Dropout(p=0.15, inplace=False)
  (fc): Linear(in_features=21760, out_features=64, bias=True)
  (batchnorm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

In [90]:
x = torch.randn(10, 4, 170)

In [91]:
model(x)

tensor([[0.4699],
        [0.8423],
        [1.1953],
        [1.1711],
        [0.7117],
        [0.0583],
        [1.0075],
        [0.9660],
        [0.5447],
        [0.7457]], grad_fn=<AddmmBackward0>)

In [92]:
core_promoter_elements = eu.utils.MinimalMEME('/cellar/users/aklie/projects/EUGENe/tests/_data/datasets/jores21/CPEs.meme')
tf_groups = eu.utils.MinimalMEME('/cellar/users/aklie/projects/EUGENe/tests/_data/datasets/jores21/TF-clusters.meme')
all_motifs = {**core_promoter_elements.motifs, **tf_groups.motifs}
len(all_motifs)

78

In [93]:
model.biconv.kernels[0][0]

tensor([[ 5.8025e-02,  2.6416e-02, -3.8535e-02, -1.0577e-02,  4.3315e-02,
          2.1613e-02, -1.7688e-02, -5.0108e-02,  4.6055e-02, -2.9319e-02,
         -3.8448e-02, -1.5185e-02, -1.7067e-03],
        [-4.1143e-03, -3.3956e-02,  6.2217e-04, -4.1170e-02,  4.2986e-02,
          2.0373e-02, -1.6676e-02,  2.3642e-02, -3.5047e-02,  4.6149e-02,
          5.6794e-03,  3.8404e-02,  6.6676e-05],
        [-2.6334e-02, -2.7681e-02, -2.8493e-02, -5.0077e-02,  4.9505e-02,
          1.5518e-02,  3.3436e-02,  4.9166e-02, -2.9328e-02,  1.6801e-02,
          4.0794e-02, -7.5571e-03,  3.2738e-02],
        [-4.1177e-02,  3.2498e-04,  4.9090e-02,  4.4912e-02,  4.4724e-03,
          1.5145e-02,  9.6194e-03,  3.7163e-02,  3.9051e-02, -3.7046e-03,
          3.5302e-02,  5.0088e-02, -2.5166e-02]])

In [94]:
eu.models.init_from_motifs(
    model, 
    all_motifs, 
    layer_name='biconv', 
    kernel_name='kernels',
    kernel_number=0
)

In [95]:
model.biconv.kernels[0][0]

tensor([[0.5100, 0.6300, 0.9960, 0.4940, 0.0400, 3.8720, 0.0080, 3.9680, 2.6120,
         3.8960, 1.3640, 2.7820, 0.5020],
        [1.5060, 1.5940, 1.2120, 2.6200, 0.0080, 0.0000, 0.0560, 0.0000, 0.0480,
         0.0000, 0.1120, 0.3260, 1.7280],
        [0.4780, 0.7960, 0.7880, 0.3020, 0.0080, 0.0000, 0.0240, 0.0080, 0.0080,
         0.0320, 0.1440, 0.4780, 1.2660],
        [1.5060, 0.9820, 1.0040, 0.5820, 3.9440, 0.1280, 3.9120, 0.0240, 1.3320,
         0.0720, 2.3820, 0.4140, 0.5020]])

In [96]:
model(x)

tensor([[1.3220],
        [0.1054],
        [1.1493],
        [0.2871],
        [1.2071],
        [0.4177],
        [1.4466],
        [0.3072],
        [0.6756],
        [0.6425]], grad_fn=<AddmmBackward0>)

In [28]:
from eugene.utils._motif import _create_kernel_matrix

In [46]:
model.__getattr__("biconv").kernels[0] = kern

In [40]:
model."biconv"

SyntaxError: invalid syntax (2592916639.py, line 1)

In [36]:
kern = model.biconv.kernels[0]

In [37]:
kern = _create_kernel_matrix((128, 4, 13), all_motifs)

In [56]:
model.conv.weight = torch.nn.Parameter(torch.randn(128, 128, 13))

In [57]:
model.conv.weight

Parameter containing:
tensor([[[-1.1160, -0.3325, -0.7744,  ...,  0.0685, -0.1617, -0.7123],
         [ 0.0985, -1.0684,  0.9452,  ...,  0.0340,  0.8200, -0.8049],
         [-0.3100, -1.2196, -0.4721,  ...,  0.7886,  0.2196,  2.7004],
         ...,
         [-1.1247, -0.0458, -0.4971,  ...,  0.4404, -1.2233, -0.1531],
         [ 0.6456, -0.5590, -1.2134,  ..., -0.5988, -0.2380, -0.2558],
         [-0.8898, -1.3874, -0.8575,  ..., -0.4385,  1.1100, -0.2351]],

        [[-0.7794,  0.8648, -0.3815,  ..., -1.6818,  0.1235,  1.6162],
         [ 0.7390, -1.7336,  0.4566,  ...,  0.1805, -0.2668,  0.9915],
         [ 0.9938, -0.0948,  1.4486,  ...,  1.2259, -0.0529,  0.5860],
         ...,
         [-0.9741, -2.5641, -0.8386,  ...,  0.6279, -2.0081, -1.0378],
         [-0.1105,  0.5527, -0.0749,  ..., -0.5626, -0.4269,  0.9372],
         [-0.6103,  2.6745,  0.2042,  ..., -1.4715, -0.0636,  0.8312]],

        [[-1.8713,  0.9585,  1.2868,  ...,  0.8011,  0.1002,  0.1813],
         [-0.8813, -0.4

In [104]:
core_promoter_elements = MinimalMEME('/cellar/users/aklie/projects/EUGENe/tests/_data/datasets/jores21/CPEs.meme')
tf_groups = MinimalMEME('/cellar/users/aklie/projects/EUGENe/tests/_data/datasets/jores21/TF-clusters.meme')
all_motifs = {**core_promoter_elements.motifs, **tf_groups.motifs}

In [122]:
kernel = torch.zeros(128, 4, 13)
nn.init.xavier_uniform_(kernel)

# overwrite part of kernel with pfms from motifs
for i, motif_id in enumerate(all_motifs):
    motif = all_motifs[motif_id]
    # convert PFM to PWM, assume equal background frequency of 0.25
    # truncates motifs longer than 13bp to 13bp
    kernel[i, :, :min(len(motif), kernel.shape[2])] = torch.tensor(motif.pfm[:min(len(motif), kernel.shape[2]), :] / 0.25).transpose(0, 1)

In [124]:
model.biconv.kernels[0] = kernel

In [125]:
model(x)

tensor([[-0.0013],
        [ 0.0682],
        [ 0.0221],
        [-0.0021],
        [ 0.0844],
        [ 0.1157],
        [ 0.0566],
        [ 0.2570],
        [-0.1326],
        [-0.1769]], grad_fn=<AddmmBackward0>)

In [18]:
core_promoter_elements = eu.utils.MinimalMEME('/cellar/users/aklie/projects/EUGENe/tests/_data/datasets/jores21/CPEs.meme')
tf_groups = eu.utils.MinimalMEME('/cellar/users/aklie/projects/EUGENe/tests/_data/datasets/jores21/TF-clusters.meme')
all_motifs = {**core_promoter_elements.motifs, **tf_groups.motifs}

AttributeError: module 'eugene.utils' has no attribute 'MinimalMEME'

# Load from config

In [47]:
model_config = "../_configs/test_fcn.yaml"
eu.models.load_config("FCN", model_config)

FCN(
  (fcn): BasicFullyConnectedModule(
    (module): Sequential(
      (0): Linear(in_features=264, out_features=128, bias=True)
      (1): ReLU(inplace=True)
      (2): Linear(in_features=128, out_features=64, bias=True)
      (3): ReLU(inplace=True)
      (4): Linear(in_features=64, out_features=1, bias=True)
    )
  )
  (r_squared): R2Score()
)

In [40]:
test_dict = {'input_len': 66, 'strand': 'ss', 'fc_kwargs': {'output_dim': 1, 'hidden_dims': [128, 66]}}

In [44]:
out = "../_configs/test_fcn.yaml"

In [42]:
test_dict

{'input_len': 66,
 'strand': 'ss',
 'fc_kwargs': {'output_dim': 1, 'hidden_dims': [128, 66]}}

---